# Issue 139 Notebook Dricksvatten nära SAT 

* denna [Notebook](https://github.com/salgo60/Stockholm_Archipelago_Trail/blob/main/notebook/Issue_139_Notebook_Dricksvatten_n%C3%A4ra_SAT.ipynb)
* [Issue 139](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/139) 

Jämför 
* [Issue 132 Toaletter](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/139) 


Output 
* [kartor/Issue_139_dricksvatten_nara_stockholm_archipelago_trail_2025_08_17_23_13.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_139_dricksvatten_nara_stockholm_archipelago_trail_2025_08_17_23_13.html)
* [kartor/Issue_139_dricksvatten_nara_stockholm_archipelago_trail_2025_08_18_19_39.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_139_dricksvatten_nara_stockholm_archipelago_trail_2025_08_18_19_39.html)
* [kartor/Issue_139_dricksvatten_nara_stockholm_archipelago_trail_2025_08_18_19_49.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_139_dricksvatten_nara_stockholm_archipelago_trail_2025_08_18_19_49.html)
* [kartor/sat_dricksvatten_table_2025_08_19_13_53.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/sat_dricksvatten_table_2025_08_19_13_53.html)
* [kartor/Issue_139_dricksvatten_nara_stockholm_archipelago_trail_2025_08_27_12_40.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_139_dricksvatten_nara_stockholm_archipelago_trail_2025_08_27_12_40.html)




version 0.1

In [1]:
import time
from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))


Start: 2025-09-20 19:43:01


In [12]:
# !pip install geopandas shapely folium requests SPARQLWrapper --quiet

import os, re, requests
from urllib.parse import quote
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString, MultiLineString, Point, mapping
from SPARQLWrapper import SPARQLWrapper, JSON
from collections import defaultdict
import folium
from folium import Marker, Icon, FeatureGroup, LayerControl, Popup
from datetime import datetime
from branca.element import Template, MacroElement

# =========================
# Hjälpfunktion: About-box
# =========================
from branca.element import Template, MacroElement

from folium import Element  # alias of branca.element.Element

def add_about_box(m, issue_number: int, map_name: str, generated_ts: str):
    html = f"""
    <div id="about-box" style="
        position: fixed; bottom: 18px; right: 18px; z-index: 99999;
        background: rgba(255,255,255,0.98); border: 1px solid #ddd; border-radius: 10px;
        padding: 10px 12px; box-shadow: 0 2px 10px rgba(0,0,0,.15); max-width: 280px;
        font-family: system-ui, -apple-system, Segoe UI, Roboto, Helvetica, Arial, sans-serif; font-size: 12px;">
      <div style="font-weight:700; margin-bottom: 4px;">Om: {map_name}</div>
      <div style="line-height:1.35;">
        Genererad: {generated_ts}<br>
        Ärende: <a href="https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/{issue_number}"
                   target="_blank">GitHub Issue #{issue_number}</a><br>
        Datakällor: Wikidata, OSM/Overpass
      </div>
    </div>
    """
    m.get_root().html.add_child(Element(html))




# =========================
# 1) Hämta SAT-etapper via Wikidata
# =========================
print("🔍 Hämtar SAT-etapper från Wikidata...")
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery("""
SELECT ?item ?itemLabel ?islandLabel ?osmid WHERE {
  ?item wdt:P361 wd:Q131318799;
        wdt:P31 wd:Q2143825;
        wdt:P402 ?osmid.
  OPTIONAL { ?item wdt:P706 ?island. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

etapper = []
for r in results["results"]["bindings"]:
    etapper.append({
        "id": r["osmid"]["value"],
        "label": r.get("itemLabel", {}).get("value", ""),
        "island": r.get("islandLabel", {}).get("value", "")
    })
osm_ids = [e['id'] for e in etapper]
print(f"✅ Hittade {len(osm_ids)} etapper med OSM-relationer")

# =========================
# 2) Hämta geometrier per relation via Overpass
# =========================
print("📡 Hämtar geometrier från Overpass (per relation)...")
overpass_url = "http://overpass-api.de/api/interpreter"
geom_per_rel = {}
all_lines = []

for rel_id in osm_ids:
    q = f"""
    [out:json][timeout:60];
    relation({rel_id});
    (._;>>;);
    out geom;
    """
    r = requests.post(overpass_url, data={"data": q})
    if r.status_code != 200:
        print(f"⚠️ Fel för relation {rel_id}: {r.text[:200]}...")
        continue
    rel_geoms = []
    for el in r.json().get("elements", []):
        if el.get("type") == "way" and "geometry" in el:
            coords = [(pt["lon"], pt["lat"]) for pt in el["geometry"]]
            if len(coords) >= 2:
                line = LineString(coords)
                rel_geoms.append(line)
                all_lines.append(line)
    if rel_geoms:
        geom_per_rel[rel_id] = rel_geoms

if not all_lines:
    raise ValueError("Inga geometrier hittades från OSM-relationer kopplade via Wikidata.")

# Bygg MultiLineString/LineString per etapp
meta_rows, geom_rows = [], []
for meta in etapper:
    geoms = geom_per_rel.get(meta["id"])
    if not geoms:
        print(f"⚠️ Saknar geometri för {meta['label']} (rel {meta['id']}) – hoppar över.")
        continue
    meta_rows.append(meta)
    geom_rows.append(MultiLineString(geoms) if len(geoms) > 1 else geoms[0])

gdf_trail = gpd.GeoDataFrame(geometry=all_lines, crs="EPSG:4326")
meta_gdf = gpd.GeoDataFrame(meta_rows, geometry=geom_rows, crs="EPSG:4326")
print(f"🧭 Etapper med geometri: {len(meta_gdf)}")

# =========================
# 3) Buffert 50/100/200 m (Shapely 2: union_all)
# =========================
print("🧮 Skapar 50/100/200 m-buffert...")
trail_utm = gdf_trail.to_crs(3006)
buf_50_utm  = trail_utm.buffer(50)
buf_100_utm = trail_utm.buffer(100)
buf_200_utm = trail_utm.buffer(200)

buffer_50  = buf_50_utm.to_crs(4326).union_all()
buffer_100 = buf_100_utm.to_crs(4326).union_all()
buffer_200 = buf_200_utm.to_crs(4326).union_all()

# =========================
# 4) Hämta dricksvatten (nwr + out center)
# =========================
bbox = gdf_trail.total_bounds  # [minx, miny, maxx, maxy]
q_water = f"""
[out:json][timeout:60];
(
  nwr["amenity"="drinking_water"]({bbox[1]},{bbox[0]},{bbox[3]},{bbox[2]});
  nwr["amenity"="toilets"]["drinking_water"="yes"]({bbox[1]},{bbox[0]},{bbox[3]},{bbox[2]});
  nwr["amenity"="water_point"]["drinking_water"="yes"]({bbox[1]},{bbox[0]},{bbox[3]},{bbox[2]});
);
out center;
"""
print("💧 Hämtar dricksvatten (nwr) från Overpass...")
r = requests.post(overpass_url, data={"data": q_water})
r.raise_for_status()
elements = r.json().get("elements", [])

waters = []
for el in elements:
    tags = el.get("tags", {})
    typ = el.get("type")
    if typ == "node":
        lon, lat = el["lon"], el["lat"]
    else:
        center = el.get("center")
        if not center:
            continue
        lon, lat = center["lon"], center["lat"]
    waters.append({
        "geometry": Point(lon, lat),
        "tags": tags,
        "id": el["id"],
        "osm_type": typ,
        "water_sites": 1,
    })
gdf_water = gpd.GeoDataFrame(waters, crs="EPSG:4326")
print(f"✅ Hittade {len(gdf_water)} dricksvatten-objekt inom bbox")

# =========================
# 5) Filtrera till 50/100/200 m
# =========================
in_range_50  = gdf_water[gdf_water.geometry.covered_by(buffer_50)]
in_range_100 = gdf_water[gdf_water.geometry.covered_by(buffer_100)]
in_range_200 = gdf_water[gdf_water.geometry.covered_by(buffer_200)]
print(f"✅ {len(in_range_50)} inom 50 m, {len(in_range_100)} inom 100 m, {len(in_range_200)} inom 200 m")

# =========================
# 6) Närmaste etapp per dricksvatten (för varje avstånd)
# =========================
meta_utm = meta_gdf.to_crs(3006)

def nearest_join(df):
    if df.empty:
        return df
    wutm = df.to_crs(3006)
    joined = gpd.sjoin_nearest(
        wutm,
        meta_utm[["label", "island", "geometry"]],
        how="left",
        distance_col="distance_m"
    ).to_crs(4326)
    return joined

joined_50  = nearest_join(in_range_50)
joined_100 = nearest_join(in_range_100)
joined_200 = nearest_join(in_range_200)

# =========================
# 7) Summary (200 m som standard)
# =========================
summary = (
    joined_200.groupby(["label", "island"], as_index=False)
    .agg(sites=("geometry", "count"), avg_distance_m=("distance_m", "mean"))
    .sort_values(["sites"], ascending=[False])
)
print("📊 Summary drinking water (200 m):")
print(summary.head(1000))

# =========================
# 8) Spara filer + Folium-karta
# =========================
timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M")
timestamp_human = datetime.now().strftime("%Y-%m-%d %H:%M")

os.makedirs("../kartor", exist_ok=True)

# Spara summary
summary_csv = f"../kartor/sat_dricksvatten_summary_{timestamp}.csv"
summary.to_csv(summary_csv, index=False)

# Spara dricksvatten inom 200 m (GeoJSON + CSV)
waters_geojson = f"../kartor/sat_dricksvatten_inrange_{timestamp}.geojson"
waters_csv = f"../kartor/sat_dricksvatten_inrange_{timestamp}.csv"
in_range_200[["id", "osm_type", "tags", "geometry"]].to_file(waters_geojson, driver="GeoJSON")
in_range_200.drop(columns="geometry").to_csv(waters_csv, index=False)

# Bygg karta
center = gdf_trail.geometry.union_all().centroid
m = folium.Map(location=[center.y, center.x], zoom_start=9, control_scale=True)

# Etapper (hoppa över 3 specifika)
skip_labels = {
    "SAT Arholma", "SAT Brottö", "SAT Lidö",
    "SAT Landsort", "SAT Grinda", "SAT Nämndö", "SAT Nåttarö", "SAT Rånö",
    "SAT Ålö", "SAT Utö", "SAT Ornö", "SAT Fjärdlång", "SAT Svartsö",
    "SAT Runmarö", "SAT Ingmarsö", "SAT Finnhamn", "SAT Yxlan",
    "SAT Möja", "SAT Furusund", "SAT Sandhamn"
}

# when adding stage layers:
for i, row in meta_gdf.reset_index(drop=True).iterrows():
    if str(row["label"]).strip() in skip_labels:
        continue
    ...

colors = [
    "blue","green","purple","orange","darkred","cadetblue","lightgray","darkblue",
    "darkgreen","pink","lightblue","lightgreen","gray","black","beige","lightred"
]
for i, row in meta_gdf.reset_index(drop=True).iterrows():
    if str(row["label"]).strip() in skip_labels:
        continue
    color = colors[i % len(colors)]
    popup = f"<b>{row['label']}</b><br>Ö: {row['island']}"
    folium.GeoJson(
        data=mapping(row.geometry),
        name=row["label"],
        style_function=lambda x, c=color: {"color": c, "weight": 3}
    ).add_child(folium.Popup(popup, max_width=350)).add_to(m)

# Buffertlager 50/100/200
folium.GeoJson(
    data=mapping(buffer_50),
    name="50 m Buffert",
    style_function=lambda x: {'fillColor': '#0000ff', 'color': '#0000ff', 'weight': 1, 'fillOpacity': 0.05}
).add_to(m)
folium.GeoJson(
    data=mapping(buffer_100),
    name="100 m Buffert",
    style_function=lambda x: {'fillColor': '#0000ff', 'color': '#0000ff', 'weight': 1, 'fillOpacity': 0.07}
).add_to(m)
folium.GeoJson(
    data=mapping(buffer_200),
    name="200 m Buffert",
    style_function=lambda x: {'fillColor': '#0000ff', 'color': '#0000ff', 'weight': 1, 'fillOpacity': 0.1}
).add_to(m)

# === Hjälpfunktioner för popup ===
def _split_multi(s: str):
    return [p.strip() for p in re.split(r'[;|]', s) if p.strip()]

def commons_thumb_html(value: str, width: int = 300) -> str:
    v = value.strip()
    lower = v.lower()
    if lower.startswith("category:"):
        url = f"https://commons.wikimedia.org/wiki/{quote(v.replace(' ', '_'))}"
        return f'<a href="{url}" target="_blank">{v}</a>'
    if any(lower.startswith(p) for p in ("file:", "image:", "media:")):
        filename = v.split(":", 1)[1]
    else:
        filename = v
    fn_enc = quote(filename.replace(" ", "_"))
    img = f"https://commons.wikimedia.org/wiki/Special:FilePath/{fn_enc}?width={width}"
    page = f"https://commons.wikimedia.org/wiki/File:{fn_enc}"
    return f'<a href="{page}" target="_blank"><img src="{img}" style="max-width:{width}px"></a>'

def link_commons_title(value: str) -> str:
    url = f"https://commons.wikimedia.org/wiki/{quote(value.replace(' ', '_'))}"
    return f'<a href="{url}" target="_blank">{value}</a>'

def link_wikidata(qid: str) -> str:
    if not qid:
        return ""
    q = qid.strip().upper()
    if q.startswith("Q") and q[1:].isdigit():
        return f'<a href="https://www.wikidata.org/wiki/{q}" target="_blank">{q}</a>'
    return q

def images_html_from_tags(tags: dict, max_thumbs: int = 2, thumb_width: int = 300) -> str:
    candidates = []
    for k, v in tags.items():
        if v and (k == "image" or k.startswith("image:")):
            candidates.append(v)
    wc = tags.get("wikimedia_commons")
    if wc:
        candidates.extend(_split_multi(wc))
    html_parts = []
    for val in candidates:
        val = val.strip()
        if not val:
            continue
        if val.startswith("http://") or val.startswith("https://"):
            html_parts.append(f'<a href="{val}" target="_blank"><img src="{val}" style="max-width:{thumb_width}px"></a>')
        else:
            html_parts.append(commons_thumb_html(val, width=thumb_width))
        if len(html_parts) >= max_thumbs:
            break
    return "<br>".join(html_parts)

def build_water_layer(joined_gdf, name):
    fg = FeatureGroup(name=name)
    if joined_gdf is None or joined_gdf.empty:
        return fg
    for _, r in joined_gdf.to_crs(4326).iterrows():
        tags = r.get("tags", {}) or {}
        osm_path = ('node' if r['osm_type'] == 'node'
                    else 'way' if r['osm_type'] == 'way'
                    else 'relation')
        osm_url = f"https://www.openstreetmap.org/{osm_path}/{r['id']}"
        pics_html = images_html_from_tags(tags, max_thumbs=2, thumb_width=300)
        note_sv = tags.get("note")
        note_en = tags.get("note:en")
        commons_entries = _split_multi(tags.get("wikimedia_commons", "")) if tags.get("wikimedia_commons") else []
        commons_links = ", ".join(link_commons_title(x) for x in commons_entries) if commons_entries else ""
        operator = tags.get("operator")
        operator_wd = link_wikidata(tags.get("operator:wikidata", "")) if tags.get("operator:wikidata") else ""
        access = tags.get("access")

        popup_html = f"""
        <b><a href="{osm_url}" target="_blank">OSM-objekt ({r['osm_type']})</a></b><br>
        Etapp: <b>{r.get('label','')}</b> (Ö: {r.get('island','')})<br>
        Avstånd: ~{round(float(r.get('distance_m', 0) or 0), 1)} m<br>
        """
        if pics_html: popup_html += pics_html + "<br>"
        if note_sv:   popup_html += f"Not: {note_sv}<br>"
        if note_en:   popup_html += f"Note (en): {note_en}<br>"
        if commons_links: popup_html += f"Wikimedia Commons: {commons_links}<br>"
        if operator:  popup_html += f"Operatör: {operator}<br>"
        if operator_wd: popup_html += f"Operatör (Wikidata): {operator_wd}<br>"
        if access:    popup_html += f"Access: {access}<br>"

        Marker(
            location=[r.geometry.y, r.geometry.x],
            popup=Popup(popup_html, max_width=360),
            icon=Icon(color="green", icon="tint")
        ).add_to(fg)
    return fg

# Dricksvattenlager (50/100/200 m)
drinking_50  = build_water_layer(joined_50,  "Dricksvatten inom 50 m")
drinking_100 = build_water_layer(joined_100, "Dricksvatten inom 100 m")
drinking_200 = build_water_layer(joined_200, "Dricksvatten inom 200 m")

drinking_50.add_to(m)
drinking_100.add_to(m)
drinking_200.add_to(m)

# About-box
add_about_box(m, issue_number=139, map_name="Dricksvatten nära leden", generated_ts=timestamp_human)

LayerControl(collapsed=False).add_to(m)

# Spara kartor
map_html_ts = f"../kartor/Issue_139_dricksvatten_nara_stockholm_archipelago_trail_{timestamp}.html"
map_html_latest = "../kartor/Issue_139_dricksvatten_nara_stockholm_archipelago_trail_latest.html"
m.save(map_html_ts)
m.save(map_html_latest)

print("✅ Klart!")
print(f"• Summary CSV: {summary_csv}")
print(f"• Dricksvatten GeoJSON (200 m): {waters_geojson}")
print(f"• Dricksvatten CSV (200 m): {waters_csv}")
print(f"• Karta: {map_html_ts}")
print(f"• Karta (latest): {map_html_latest}")


🔍 Hämtar SAT-etapper från Wikidata...
✅ Hittade 20 etapper med OSM-relationer
📡 Hämtar geometrier från Overpass (per relation)...
🧭 Etapper med geometri: 20
🧮 Skapar 50/100/200 m-buffert...
💧 Hämtar dricksvatten (nwr) från Overpass...
✅ Hittade 180 dricksvatten-objekt inom bbox
✅ 55 inom 50 m, 60 inom 100 m, 72 inom 200 m
📊 Summary drinking water (200 m):
            label     island  sites  avg_distance_m
3      SAT Grinda     Grinda     22       51.923936
14        SAT Utö        Utö      9       32.184136
1    SAT Finnhamn   Finnhamn      7       29.926556
5    SAT Landsort        Öja      6       14.234741
11       SAT Rånö       Rånö      5        8.879014
0     SAT Arholma    Arholma      4       52.560755
9     SAT Nåttarö    Nåttarö      3       71.325716
15        SAT Ålö        Ålö      3       40.956089
6        SAT Lidö       Lidö      2       37.572461
7        SAT Möja       Möja      2       55.266851
8       SAT Nämdö      Nämdö      2       11.054254
10       SAT Ornö 

In [13]:
import math, time
NOTES_API = "https://api.openstreetmap.org/api/0.6/notes.json"

def _deg_buffer(lat: float, radius_m: float):
    dlat = radius_m / 111_320.0
    coslat = max(0.01, math.cos(math.radians(lat)))
    dlon = dlat / coslat
    return dlat, dlon

def get_osm_notes_nearby(lat: float, lon: float, radius_m: float = 50, limit: int = 50, sleep_s: float = 0.0):
    dlat, dlon = _deg_buffer(lat, radius_m)
    bbox = f"{lon-dlon:.6f},{lat-dlat:.6f},{lon+dlon:.6f},{lat+dlat:.6f}"
    params = {"bbox": bbox, "limit": str(limit), "closed": "no", "sort": "created_at"}
    try:
        resp = requests.get(NOTES_API, params=params, timeout=25)
        if sleep_s: time.sleep(sleep_s)
        resp.raise_for_status()
        data = resp.json()
    except Exception as e:
        return [{"id": None, "status": "error", "last_text": f"(notes API-fel: {e})", "last_date": None}]
    out = []
    for feat in data.get("features", []):
        props = feat.get("properties", {})
        comments = props.get("comments", [])
        last_text, last_date = (comments[-1].get("text"), comments[-1].get("date")) if comments else (None, None)
        out.append({"id": props.get("id"), "status": props.get("status"), "last_text": last_text, "last_date": last_date})
    return out


In [8]:
# =========================
# 9) Tabell över alla dricksvattenobjekt med extra metadata
# =========================
rows = []
for i, r in joined.iterrows():
    tags = r.get("tags", {}) or {}
    osm_num = f"{r['osm_type']}/{r['id']}"
    lat, lon = r.geometry.y, r.geometry.x

    # --- OSM Notes inom 50 m via OSM Notes API ---
    notes = get_osm_notes_nearby(lat, lon, radius_m=50, limit=50, sleep_s=0.0)
    note_texts  = [n["last_text"] for n in notes if n.get("last_text")]
    note_ids    = [str(n["id"]) for n in notes if n.get("id")]
    note_links  = [f"https://www.openstreetmap.org/note/{nid}" for nid in note_ids]

    # --- Operator & felanmälan ---
    operator = tags.get("operator")
    operator_wd = tags.get("operator:wikidata")
    contact_url = tags.get("contact:website") or tags.get("operator:website")
    contact_phone = tags.get("contact:phone")

    # --- Wikimedia Commons ---
    commons = tags.get("wikimedia_commons")

    # --- Senaste check ---
    check_date = tags.get("check_date")
    start_date = tags.get("start_date")  # om den finns

    row = {
        "number": i + 1,
        "OSM Number": osm_num,
        "Wikidata Number": tags.get("wikidata") or operator_wd,
        "label": r.get("label"),
        "island": r.get("island"),
        "access": tags.get("access"),
        "drinking_water": tags.get("drinking_water"),
        "note": tags.get("note"),
        "note:en": tags.get("note:en"),
        "operator": operator,
        "operator:wikidata": operator_wd,
        "contact_url": contact_url,
        "contact_phone": contact_phone,
        "wikimedia_commons": commons,
        "image": tags.get("image"),
        "image:license": tags.get("image:license"),
        "image:license:wikidata": tags.get("image:license:wikidata"),
        "check_date": check_date,
        "start_date": start_date,
        "osm_notes_nearby": " | ".join(note_texts) if note_texts else "",
        "osm_note_ids": ", ".join(note_ids) if note_ids else "",
        "osm_note_links": ", ".join(note_links) if note_links else "",
    }

    # --- Fixme-logik ---
    fixme = []
    if not tags.get("drinking_water"):
        fixme.append("saknar drinking_water-tag")
    elif (tags.get("drinking_water") or "").lower() == "unknown":
        fixme.append("dricksvatten ej testat / unknown")
    if not tags.get("access"):
        fixme.append("saknar access")
    if not commons and not tags.get("image"):
        fixme.append("saknar bild")
    if tags.get("image") and not (tags.get("image:license") or tags.get("image:license:wikidata")):
        fixme.append("bild utan licensinfo")
    if not operator:
        fixme.append("saknar operator")
    if not (tags.get("wikidata") or operator_wd):
        fixme.append("saknar länk till Wikidata")
    if not check_date:
        fixme.append("saknar check_date")
    if note_texts:
        fixme.append("kolla närliggande OSM Note")

    row["fixme"] = "; ".join(fixme) if fixme else ""
    rows.append(row)

df_water = pd.DataFrame(rows)

print("✅ Klart!")

# Spara som CSV + HTML + Markdown
table_csv = f"../kartor/sat_Issue_139_dricksvatten_table_{timestamp}.csv"
print(f"• Table_CSV: {table_csv}")

df_water.to_csv(table_csv, index=False)

table_html = f"../kartor/sat_Issue_139_dricksvatten_table_{timestamp}.html"
print(f"• Table_htlm: {table_html}")
df_water.to_html(table_html, index=False, escape=False)

table_md = f"../kartor/sat_Issue_139_dricksvatten_table_{timestamp}.md"
print(f"• Table_md: {table_md}")
with open(table_md, "w", encoding="utf-8") as f:
    f.write(df_water.head(20).to_markdown(index=False))


NameError: name 'joined' is not defined

In [ ]:
from itables import init_notebook_mode, show
from itables import options as itbl_options

init_notebook_mode(all_interactive=True)
itbl_options.warn_on_undocumented_option = False  

# visa upp till 200 rader
itbl_options.lengthMenu = [10, 25, 50, 100, 200, -1]  # -1 = alla
itbl_options.pageLength = 200
#itbl_options.dom = 'lfrtip'
#itbl_options.searchHighlight = True

show(df_water, scrollX=True, autoWidth=True)


In [ ]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("Total time elapsed: {:.2f} seconds".format(elapsed_time))